In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('course_lead_scoring.csv')

In [3]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


# Data preparation

In [5]:
# Check if the missing values are presented in the features.
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [ ]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [7]:
# * If there are missing values:
#     * For caterogiral features, replace them with 'NA'
#     * For numerical features, replace with with 0.0 
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna('NA')
    else:
        df[col] = df[col].fillna(0)

In [8]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [9]:
### Question 1
# What is the most frequent observation (mode) for the column `industry`?
df['industry'].mode()[0]

'retail'

In [12]:
### Question 2

# Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset. 
# In a correlation matrix, you compute the correlation coefficient between every pair of features.
df.select_dtypes(include=['float64', 'int64']).corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [13]:
### Split the data

# * Split your data in train/val/test sets with 60%/20%/20% distribution.
# * Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
# * Make sure that the target value `y` is not in your dataframe.
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [ ]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [15]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [16]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [18]:
df_train.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,retail,0,58472.0,student,middle_east,5,0.03
1,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77
2,paid_ads,technology,3,81973.0,employed,north_america,2,0.59
3,NA,technology,1,74956.0,employed,europe,3,0.34
4,organic_search,retail,3,59335.0,student,australia,1,0.98


In [19]:
### Question 3
# * Calculate the mutual information score between `y` and other categorical variables in the dataset. Use the training set only.
# * Round the scores to 2 decimals using `round(score, 2)`.
from sklearn.metrics import mutual_info_score
def mutual_info_score_series(x, y):
    return mutual_info_score(x, y)

categorical_cols = list(df_train.select_dtypes(include=['object']).columns)
mi_scores = {}
for col in categorical_cols:
    mi_scores[col] = mutual_info_score_series(df_train[col], y_train)
mi_scores = pd.Series(mi_scores).sort_values(ascending=False)
mi_scores = mi_scores.round(2)

mi_scores


lead_source          0.04
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

In [ ]:
### Question 4

# * Now let's train a logistic regression.
# * Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
# * Fit the model on the training dataset.
#     - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
#     - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
# * Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [28]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [21]:
dv = DictVectorizer(sparse=False)
train_dicts = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [23]:
X_train[0]

array([5.8472e+04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 5.0000e+00,
       3.0000e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00])

In [24]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [25]:
val_dicts = df_val.to_dict(orient='records')

In [26]:
X_val = dv.transform(val_dicts)

In [27]:
y_pred = model.predict(X_val)

In [29]:
accuracy = accuracy_score(y_val, y_pred)
round(accuracy, 2)

0.7

In [41]:
### Question 5 

# * Let's find the least useful feature using the *feature elimination* technique.
# * Train a model using the same features and parameters as in Q4 (without rounding).
# * Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
# * For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

In [42]:
dv = DictVectorizer(sparse=False)
train_dicts = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [43]:
X_train[0]

array([5.8472e+04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 5.0000e+00,
       3.0000e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00])

In [44]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [45]:
val_dicts = df_val.to_dict(orient='records')

In [46]:
X_val = dv.transform(val_dicts)

In [47]:
y_pred = model.predict(X_val)

In [48]:
accuracy = accuracy_score(y_val, y_pred)
accuracy

0.6996587030716723

In [49]:
features_to_check = ['industry', 'employment_status', 'lead_score']
results = {}

for feature in features_to_check:
    # Prepare the data without the feature
    df_train_reduced = df_train.drop(columns=[feature])
    df_val_reduced = df_val.drop(columns=[feature])
    
    dv_reduced = DictVectorizer(sparse=False)
    train_dicts_reduced = df_train_reduced.to_dict(orient='records')
    X_train_reduced = dv_reduced.fit_transform(train_dicts_reduced)
    
    model_reduced = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model_reduced.fit(X_train_reduced, y_train)
    
    val_dicts_reduced = df_val_reduced.to_dict(orient='records')
    X_val_reduced = dv_reduced.transform(val_dicts_reduced)
    
    y_pred_reduced = model_reduced.predict(X_val_reduced)
    
    accuracy_reduced = accuracy_score(y_val, y_pred_reduced)
    
    # Calculate the difference in accuracy
    accuracy_diff = accuracy - accuracy_reduced
    results[feature] = accuracy_diff

In [50]:
results

{'industry': 0.0,
 'employment_status': 0.0034129692832763903,
 'lead_score': -0.0068259385665528916}

In [73]:
### Question 6

# * Now let's train a regularized logistic regression.
# * Let's try the following values of the parameter `C`: `[0.01, 0.1, 1, 10, 100]`.
# * Train models using all the features as in Q4.
# * Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

In [80]:
dv = DictVectorizer(sparse=False)
train_dicts = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)

c_parameters = [0.01, 0.1, 1, 10, 100]

results = {}
for c in c_parameters:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    print(model)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    results[c] = round(accuracy, 3)

results

LogisticRegression(C=0.01, max_iter=1000, random_state=42, solver='liblinear')
LogisticRegression(C=0.1, max_iter=1000, random_state=42, solver='liblinear')
LogisticRegression(C=1, max_iter=1000, random_state=42, solver='liblinear')
LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')
LogisticRegression(C=100, max_iter=1000, random_state=42, solver='liblinear')


{0.01: 0.7, 0.1: 0.7, 1: 0.7, 10: 0.7, 100: 0.7}